<a href="https://colab.research.google.com/github/romeodiaz/colabgoogle/blob/main/%5BMAKE_A_COPY%5D_CVIP_%5BGE_AIVE%5D_ADK_Deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Authenticate GCP
from google.colab import auth
auth.authenticate_user()

In [ ]:
# @title Clone GitHub Repo
!git clone <github-repo>
%cd <repo-folder>
!git checkout <branch-name>

Cloning into 'ceb-agent'...
remote: Enumerating objects: 303, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 303 (delta 0), reused 1 (delta 0), pack-reused 295 (from 1)
Receiving objects: 100% (303/303), 29.46 MiB | 31.53 MiB/s, done.
Resolving deltas: 100% (211/211), done.
/content/ceb-agent
Branch '0.1.4-tracking' set up to track remote branch '0.1.4-tracking' from 'origin'.
Switched to a new branch '0.1.4-tracking'
/content/ceb-agent/ceb-agent


In [ ]:
# @title Install AI Platform Packages
!pip install -q google-cloud-aiplatform[adk,agent_engines]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 6.5 MB/s eta 0:00:00


In [ ]:
# @title Initialize Vertex AI
import vertexai

PROJECT_ID = "cvip-16562"
LOCATION = "us-central1"

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket='gs://<bucket-name>',
)

In [ ]:
# @title Initialize agent.py (root agent)
from agent import root_agent
from vertexai.preview import reasoning_engines

app = reasoning_engines.AdkApp(
    agent=root_agent,
    # enable_tracing=True,
)

/content/ceb-agent/ceb-agent/agent.py:152: SyntaxWarning: invalid escape sequence '\`'
  - **CRITICAL - Minified JSON Output Only:** Your final output must be a valid, single-line, minified JSON object. Do not wrap it in markdown code fences (\`\`\`json), add any text before or after it, or include newline characters.


In [ ]:
# @title Test create_session
session = app.create_session(user_id="u_123")
print(f'Create Session:\n{session}\n')
print(f'List Sessions:\n{app.create_session(user_id="u_123")}\n')
print(f'Get Session:\n{app.get_session(user_id="u_123", session_id=session.id)}')

Create Session:
id='8116d15e-ef79-44ab-b0d1-f8347ba09b8f' app_name='default-app-name' user_id='u_123' state={} events=[] last_update_time=1767129119.473641

List Sessions:
id='c847b24b-ccab-43f3-b8c4-108c25f5a0a0' app_name='default-app-name' user_id='u_123' state={} events=[] last_update_time=1767129119.4751692

Get Session:
id='8116d15e-ef79-44ab-b0d1-f8347ba09b8f' app_name='default-app-name' user_id='u_123' state={} events=[] last_update_time=1767129119.473641


/usr/local/lib/python3.12/dist-packages/vertexai/preview/reasoning_engines/templates/adk.py:804: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  self._tmpl_attrs["credential_service"] = InMemoryCredentialService()
/usr/local/lib/python3.12/dist-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()


In [ ]:
# @title Test company_profile generation
import json
message = 'hello'

for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message=message,
):
    print(event)

{'model_version': 'gemini-2.5-pro', 'content': {'parts': [{'function_call': {'id': 'adk-2aee2b78-49cc-44ac-91b3-4be27bd56817', 'args': {'agent_name': 'helper_sub_agent'}, 'name': 'transfer_to_agent'}, 'thought_signature': 'CuUCAY89a18t5bytMyi4WnrSIOTw3BefmuhTxJNCOFGD3eD_yAemblG3ykGkfD2ct0QSPutVWCUpk865vtgz0L5NVIS7Okchw1CmZi1W3BkTIdL5ix7jXSkly37FBrLPB6wviAjbbd_WyqVfPs49p2S7SA_280AIbTIrfS06lgIO_mzijnTxuRtA4DNgjab9EksreRkaZeJIErLOhyW43QJDD1bWvcsGg8eEOYTWx2pAXIq7XNgknu9qHsGPUhrENdX_VrqittVzeALxkA1_tJn6-pSrXDqPub6dbDdP9W1ql3CnLDTXI03CYY04Gloc6r85d9etW3__W_Q64oCEhKhUU7z9F8Tp4ussg2U7SjHoqSihnxvPjETOUjy_JSxAnit8J1I0Q-xvldQuNT_W9cQrANq7p4U18ZbKOtLvImmBppVkRco3Lvq6wfc7i49E5e1cz35TcQEpCQDQpBhRXFIB5bonNlzu'}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {'candidates_token_count': 13, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 13}], 'prompt_token_count': 2638, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 2638}], 'thoughts_token_count': 68, 't

In [ ]:
# @title Update EXISTING Agent Engine
from vertexai import agent_engines
from agent import root_agent # REMOVE IF INITIALIZING AGENT.PY

remote_app = agent_engines.update(
    resource_name='projects/866863017863/locations/us-central1/reasoningEngines/xxxxxxxxxxxxx',
    agent_engine=root_agent,
    description='<agent-description>',
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]"
    ],
    extra_packages=[
        "agent.py",
        "config.py",
        "callbacks.py",
        "prompts.py",
        "utils.py",
        "tools.py",
        "sheets_data.py",
        "ceb-embeddings-12162025.jsonl",
        "ceb-definitions-12162025.json",
        "tracking/aive_activity_client.py",
        "tracking/aive_logging_client.py",
        "tracking/aive_tracking_client.py"
    ]
)

INFO:vertexai.agent_engines:Identified the following requirements: {'pydantic': '2.12.3', 'google-cloud-datastore': '2.21.0', 'google-cloud-monitoring': '2.28.0', 'google-colab': '1.0.0', 'google-cloud-spanner': '3.60.0', 'grpc-google-iam-v1': '0.14.3', 'google-cloud-appengine-logging': '1.7.0', 'cloudpickle': '3.1.2', 'google-cloud-discoveryengine': '0.13.12', 'google-cloud-speech': '2.34.0', 'google-cloud-core': '2.5.0', 'google-cloud-logging': '3.12.1', 'google-cloud-audit-log': '0.4.0', 'google-cloud-bigtable': '2.34.0', 'google-cloud-dataproc': '5.23.0', 'google-cloud-functions': '1.21.0', 'googleapis-common-protos': '1.72.0', 'google-cloud-translate': '3.23.0', 'pydantic_core': '2.41.4', 'google-generativeai': '0.8.5', 'google-api-core': '2.28.1', 'google-cloud-bigquery-connection': '1.19.0', 'google-cloud-secret-manager': '2.25.0', 'dataproc-spark-connect': '1.0.1', 'google-cloud-trace': '1.17.0', 'google-ai-generativelanguage': '0.6.15', 'google-cloud-pubsub': '2.34.0', 'google

In [ ]:
# @title Deploy NEW Agent Engine
from vertexai import agent_engines
from agent import root_agent # REMOVE IF INITIALIZING AGENT.PY

remote_app = agent_engines.create(
    agent_engine=root_agent,
    display_name='<agent-display-name>',
    description='<agent-description>',
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]"
    ],
    extra_packages=[
        "agent.py",
        "config.py",
        "callbacks.py",
        "prompts.py",
        "utils.py",
        "tools.py",
        "sheets_data.py",
        "ceb-embeddings-12162025.jsonl",
        "ceb-definitions-12162025.json",
        "tracking/aive_activity_client.py",
        "tracking/aive_logging_client.py",
        "tracking/aive_tracking_client.py"
    ]
)

INFO:vertexai.agent_engines:Deploying google.adk.agents.Agent as an application.
INFO:vertexai.agent_engines:Identified the following requirements: {'pydantic': '2.12.3', 'cloudpickle': '3.1.2', 'google-cloud-aiplatform': '1.130.0'}
INFO:vertexai.agent_engines:The following requirements are appended: {'pydantic==2.12.3', 'cloudpickle==3.1.2'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'google-cloud-pubsub', 'scikit-learn==1.7.1', 'pydantic==2.12.3', 'cloudpickle==3.1.2']
INFO:vertexai.agent_engines:Using bucket cep-agent-qa
INFO:vertexai.agent_engines:Wrote to gs://cep-agent-qa/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://cep-agent-qa/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://cep-agent-qa/agent_engine/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_en

In [ ]:
remote_app.resource_name

'projects/866863017863/locations/us-central1/reasoningEngines/290563539825852416'

In [ ]:
import requests

token = !gcloud auth print-access-token
token = token[0].strip()

headers = {"Authorization": f"Bearer {token}"}
data = {'class_method': 'async_create_session', 'input': {'user_id': 'testtest'}}
endpoint = f'https://us-central1-aiplatform.googleapis.com/v1/{remote_app.resource_name}:query'
res = requests.post(endpoint, headers=headers, json=data)
print(res.text)
print(res)

{
  "output": {
    "id": "6265438093143179264",
    "userId": "testtest",
    "state": {},
    "appName": "290563539825852416",
    "events": [],
    "lastUpdateTime": 1766115310.484309
  }
}

<Response [200]>


In [ ]:
import json

message = 'hello'

data = {'class_method': 'async_stream_query', 'input': {'user_id': 'testtest', 'message': message}}
endpoint = f'https://us-central1-aiplatform.googleapis.com/v1/{remote_app.resource_name}:streamQuery?alt=sse'
res = requests.post(endpoint, headers=headers, json=data)
print(res.text)
print(res)

{"model_version": "gemini-2.5-pro", "content": {"parts": [{"function_call": {"id": "adk-d38d4e3b-3c5c-4f05-b284-77e1a251572a", "args": {"agent_name": "helper_sub_agent"}, "name": "transfer_to_agent"}, "thought_signature": "Ct0OAY89a19D15jlftv0cMVg8K_NjDVPIbjl63NU_0I2jhRTpvBLzkxvTvhZccljJmle_MHhOUpugjhFvaKgYqAlQ8zvsezgUPbyg2201K_QZnsjCNXPLkcPLWyksKpbtc8XqGWZ7kGUyrbLMRt5VXbvWkCVBkYX2kUKCLrAKcnckRwRuYtoW5DZUvIEbtHHuwmmiT8A2tmot4EWJ1vxkxglIG0hKmjT94JLn-iNSaYJO5nD-jbK6p5NJrhqA5ttkKquV88NW8R9ck-2kMgNFu0gxXUZDZ8-GJkO3iAY52kiv_Iq0qUzWdHNBTCmCzRRR8SSpwe1yPvIeJtYygHIP7AdEyFbO4VPsQ-KW7B1-qI-5vQoSKr-e_1Aex-DRw9YsDpCHg7A8XxTFwBUAcfq82rLsWhLzq7Jy0BVV5IAlOlKgFqq0n2Gm1dGcI0z-jdBU_gz4930Db9UR31bznenlzRLTimm4UVm2pjcCIJm9eMFOKPz3Jsdbqb8vLjrrk_LU1QaV-nLHVD0CE80A7WzkyFVp4A-qAsFmZTD_vg_LIdKHFjzPK-M279OP9yvxRC8FQOKWM0m0c_1M0hQXjbRFRmQ7PFh-J_lkdqwEimpqvkbXReodvmueJk3h0fEI6g7MO4--PsCjYhcDlYKSdmfBEckvjDPKn9jO7D2DbC6B8QVeRLQAl7I-B8MQfjAT-Z_v_oNhQXDVc4r_i1iMbRahn58GUegqw1pjBwx-7wHKyu6GeHD4z9BRTAgIaCAaI1ldwgsPbXY3